Les paramètres des algorithmes d'apprentissage sont à fixer au mieux, notamment par des techniques de validation croisée. De même, il est souvent primordial de réduire le nombre de variables explicatives, en écartant celles qui n'ont qu'un très faible impact sur la variable à expliquer. Il s'agit de ce que l'on appelle la sélection de variables (feature selection).

# Utilisation de la variance

Commençons par réaliser une régression entre 1000 individus et 10000 variables explicatives. 

In [ ]:
from sklearn import datasets
X, y = datasets.make_regression(1000,10000)

La première option pour réduire le nombre de variables explicatives consiste à utiliser la variance via l'objet VarianceThreshold. 

En effet, les variables explicatives ayant une très faible variance ne sauraient nous permettre de cadrer avec les variations d'une variable à expliquer. 

Pour calculer la variance par colonne, on peut utiliser numpy ainsi : np.var(X, axis = 1). La limite de variance pour la sélection de variables peut, par exemple, être fixée à la médiane de ces variances :

In [ ]:
np.median(np.var(X, axis = 1))

1.0006554907941583

La sélection, en tant que telle, peut se faire ainsi :

In [ ]:
var_limite = feature_selection.VarianceThreshold(np.median(np.var(X, axis=1)))
variables_a_garder = var_limite.fit_transform(X)
variables_a_garder.shape

(1000, 4858)

On a donc réduit les variables explicatives de moitié.

L'intérêt de cette sélection de variables basée sur l'exigence de variances suffisantes pour les variables explicatives, est qu'il ne nécessite aucune information sur la variable à expliquer : on peut donc l'appliquer dans le cas non supervisé.

# Sélection de variable univariée

Nous voulons ne conserver que les features d'importance dans cette régression, ce qui peut se faire avec le module feature_selection de sklearn :

---



In [ ]:
from sklearn import feature_selection
f, p = feature_selection.f_regression(X, y)

Ci-dessus, f est le score obtenu lors d'une régression sur chaque variable prise toute seule, quand p est la p-value du test associé :

1.   Élément de liste
2.   Élément de liste



In [ ]:
f[:5]

array([1.54825640e-04, 1.97565446e-02, 2.10010539e+00, 1.29086172e+00,
       1.69460166e-01])

In [ ]:
p[:5]

array([0.99007475, 0.88824751, 0.14760312, 0.25616196, 0.68068041])

Les variables pertinentes doivent avoir une petite p-value, par exemple inférieure à 0.05 :

*   Élément de liste
*   Élément de liste



In [ ]:
import numpy as np
idx = np.arange(0, X.shape[1])
variables_a_garder = idx[p < .05]
len(variables_a_garder)

546

Nous sommes donc passés de 10000 variables explicatives à un nombre bien plus raisonnable.

1.   Élément de liste
2.   Élément de liste



# Sélection de variables par norme $\ell_1$

La norme $\ell_1$ permet de compter les variables explicatives retenues. On peut donc imaginer qu'une régression linéaire pénalisée par cette norme cherchera à être parcimonieuse, fixant les coefficients des variables peu importantes à 0. Cette idée, à la base du LASSO, permet aussi de faire de la sélection de variables.

Utilisons les données de diabète sauvegardées dans scikits-learn pour effectuer une régression linéaire :

In [ ]:
import sklearn.datasets as ds
diabetes = ds.load_diabetes()

X = diabetes.data
y = diabetes.target

Regardons tout d'abord le score MSE (validation croisée) d'une simple régression linéaire sur l'ensemble des variables explicatives de ces données :

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score,ShuffleSplit

shuff = ShuffleSplit(n_splits=10, 
                     test_size=0.25, 
                     random_state=0)
score_before = cross_val_score(lr,X,y,
                     cv=shuff,
                     scoring=make_scorer(mean_squared_error,greater_is_better=False)).mean()
score_before

-3053.3934463082664

Regardons maintenant les coefficients de la régression LASSO, dans laquelle on pénalise la régression linéaire par la norme $\ell_1$, de sorte à obtenir une compression des coefficients de régression :

In [ ]:
from sklearn.linear_model import LassoCV
lasso_cv = LassoCV()
lasso_cv.fit(X,y)
lasso_cv.coef_

array([  -0.        , -226.2375274 ,  526.85738059,  314.44026013,
       -196.92164002,    1.48742026, -151.78054083,  106.52846989,
        530.58541123,   64.50588257])

Le premier coefficient étant à 0, l'hypothèse de parcimonie nous incite à le supprimer :

In [ ]:
import numpy as np
columns = np.arange(X.shape[1])[lasso_cv.coef_ != 0]
columns

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

Regardons comment évolue le MSE sans cette première variable :

In [ ]:
score_apres = cross_val_score(lr,
                           X[:,columns],y,cv=shuff,
                           scoring=make_scorer(mean_squared_error,greater_is_better=False)).mean()
score_apres

-3033.501285928968

Ecarter la première variable explicative nous a donc permis d'améliorer notre score.

Regardons ce qui se passe sur une régression avant un grand nombre de variables explicatives non informatives :

In [ ]:
X, y = ds.make_regression(noise=5)

shuff = ShuffleSplit(n_splits=10, test_size=0.25, random_state=0)
score_before = cross_val_score(lr,X,y,cv=shuff,
scoring=make_scorer(mean_squared_error,greater_is_better=False)).mean()

lasso_cv = LassoCV()
lasso_cv.fit(X,y)

columns = np.arange(X.shape[1])[lasso_cv.coef_ != 0]
score_afterwards = cross_val_score(lr,X[:,columns],y,cv=shuff,
scoring=make_scorer(mean_squared_error,greater_is_better=False)).mean()
print("Avant :",score_before)
print("Après : ",score_afterwards)

Avant : -6932.037686392553
Après :  -20.469654478854007
